### Part 1

In [1]:
from datasets import load_dataset, load_from_disk
import os, torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_path = 'imdb_subset'
# if the path exists - load the dataset from disk, otherwise download it
if os.path.exists(dataset_path):
    subset = load_from_disk(dataset_path)
else:
    dataset = load_dataset('imdb')
    subset = dataset['train'].shuffle(seed=42).select(range(500))
    subset.save_to_disk('imdb_subset')

dataset = subset
dataset = dataset.rename_column("label", "labels")

/home/ordinary/ordinaryLib/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Part 2 - Bert

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

# Train test split
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

# Train eval split
split_dataset = train_dataset.train_test_split(test_size=0.4, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Apply the tokenization function to the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Confirm dataset sizes
print(f"Train size: {len(tokenized_train)}")
print(f"Eval size: {len(tokenized_eval)}")
print(f"Test size: {len(tokenized_test)}")

Train size: 240
Eval size: 160
Test size: 100


In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",               # Directory to save model checkpoints
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    learning_rate=2e-5,                   # Learning rate
    per_device_train_batch_size=8,        # Batch size for training
    per_device_eval_batch_size=8,         # Batch size for evaluation
    num_train_epochs=10,                  # Number of epochs
    weight_decay=0.01,                    # Weight decay
    logging_dir="./logs",                 # Directory to save logs
    logging_steps=10,                     # Log every 10 steps
    save_total_limit=2                    # Limit the number of saved checkpoints
)

/home/ordinary/ordinaryLib/myenv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
from transformers import Trainer
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)  # Get the index of the max logit
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

trainer = Trainer(
    model=model,                          # The pre-trained model to fine-tune
    args=training_args,                   # Training arguments
    train_dataset=tokenized_train,        # Training dataset
    eval_dataset=tokenized_eval,          # Evaluation dataset
    tokenizer=tokenizer,                  # Tokenizer for data preprocessing
    compute_metrics=compute_metrics       # Custom metrics function
)

model = model.to(device)

/tmp/ipykernel_3705/3864364326.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.624000,0.632325,0.675000
2,0.484200,0.371623,0.850000
3,0.459800,0.399790,0.825000
4,0.206700,0.820477,0.737500
5,0.018800,0.445779,0.875000
6,0.004400,0.412153,0.900000
7,0.011500,0.572372,0.875000
8,0.008800,0.463248,0.881250
9,0.002500,0.484999,0.875000
10,0.001900,0.508464,0.868750


TrainOutput(global_step=300, training_loss=0.179014040765663, metrics={'train_runtime': 73.9767, 'train_samples_per_second': 32.443, 'train_steps_per_second': 4.055, 'total_flos': 631466532864000.0, 'train_loss': 0.179014040765663, 'epoch': 10.0})

In [7]:
# Make predictions on the test dataset
predictions = trainer.predict(tokenized_test)

# Extract predicted logits and labels
logits = predictions.predictions
predicted_labels = np.argmax(logits, axis=1)  # Get the indices of the max logits
true_labels = predictions.label_ids

In [8]:
# Calculate accuracy on the test set
test_accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 88.00%


In [9]:
torch.cuda.empty_cache()

### Part 3 - GPT 2

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling

positive_reviews = dataset.filter(lambda example: example["labels"] == 1).select(range(100))
negative_reviews = dataset.filter(lambda example: example["labels"] == 0).select(range(100))

In [11]:
# Tokenize Reviews
def tokenize_reviews(dataset, tokenizer, max_length=150):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            padding="max_length",
            truncation=True,
            max_length=max_length
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset

In [12]:
def train_save_GPT2(model, tokenizer, tokenized_dataset, prompt='The movie was'):
    # Check which dataset we are training on
    if tokenized_dataset["labels"][0] == 0:
        sentiment = "negative"
    elif tokenized_dataset["labels"][0] == 1:
        sentiment = "positive"

    training_args = TrainingArguments(
        output_dir=f"./gpt2-{sentiment}-results",
        num_train_epochs=5,
        learning_rate=2e-5,
        per_device_train_batch_size=4,  # batch size per device during training
        per_device_eval_batch_size=4,   # batch size for evaluation
        weight_decay=0.01,              # strength of weight decay
        logging_dir='./logs',      # directory for storing logs
        logging_steps=10,
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
    train_dataset = train_test_split["train"]
    eval_dataset = train_test_split["test"]
    
    # Create a Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )
    
    # Train the model
    trainer.train()
    results = trainer.evaluate()
    print(f'Perplexity: {results["eval_loss"]}')

    # Save the model and tokenizer
    save_directory = f"./gpt2-{sentiment}-results"
    trainer.save_model(save_directory)
    tokenizer.save_pretrained(save_directory)

In [13]:
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

tokenized_positive_reviews = tokenize_reviews(positive_reviews, gpt2_tokenizer)
tokenized_negative_reviews = tokenize_reviews(negative_reviews, gpt2_tokenizer)

In [14]:
train_save_GPT2(gpt2_model, gpt2_tokenizer, tokenized_positive_reviews)

/tmp/ipykernel_3705/4217484498.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,4.030700
20,3.824700
30,3.720800
40,3.714200
50,3.594500
60,3.622200
70,3.609800
80,3.466200
90,3.469900
100,3.533400


Perplexity: 3.7762298583984375


In [15]:
train_save_GPT2(gpt2_model, gpt2_tokenizer, tokenized_negative_reviews)

/tmp/ipykernel_3705/4217484498.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,3.815300
20,3.758700
30,3.695500
40,3.549600
50,3.513600
60,3.549300
70,3.401100
80,3.552200
90,3.362800
100,3.534300


Perplexity: 3.662398099899292


### AFTER

In [16]:
positive_model = GPT2LMHeadModel.from_pretrained("gpt2-positive-results")
positive_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-positive-results")

negative_model = GPT2LMHeadModel.from_pretrained("gpt2-negative-results")
negative_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-negative-results")

In [17]:
# Parameters for text generation
max_length = 150
temperature = 0.7
top_k = 50
top_p = 0.9
repetition_penalty = 1.2
prompt = 'The movie was'

input_ids_pos = positive_tokenizer.encode(prompt, return_tensors="pt")
attention_mask = input_ids_pos.ne(positive_tokenizer.pad_token_id) 

input_ids_neg = negative_tokenizer.encode(prompt, return_tensors="pt")
attention_mask = input_ids_neg.ne(negative_tokenizer.pad_token_id)

def generate_reviews(model, tokenizer, num_reviews=5):
    reviews = []
    for _ in range(num_reviews):
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        attention_mask = input_ids.ne(tokenizer.pad_token_id)
        
        with torch.no_grad():
            output = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                do_sample=True,
                num_return_sequences=1
            )
        review = tokenizer.decode(output[0], skip_special_tokens=True)
        reviews.append(review)
    return reviews

positive_reviews = generate_reviews(positive_model, positive_tokenizer)
negative_reviews = generate_reviews(negative_model, negative_tokenizer)

with open('generated_reviews.txt', 'w') as file:
    file.write("Reviews generated by positive model:\n")
    for i, review in enumerate(positive_reviews, start=1):
        file.write(f"{i}. {review}\n")
    file.write("\nReviews generated by negative model:\n")
    for i, review in enumerate(negative_reviews, start=1):
        file.write(f"{i}. {review}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Part 4 - flan-T5

In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

In [20]:
import random
sampled_reviews = random.sample(list(dataset), 50)

In [21]:
def zero_shot_prompt(review_text):
    prompt = f"Classify the following review as positive or negative: \"{review_text}\""
    return prompt

few_shot_examples = """
Review 1: "The movie was fantastic and full of unexpected turns." -> positive
Review 2: "The movie was boring and I didn't enjoy it." -> negative
"""

def few_shot_prompt(review_text):
    prompt = few_shot_examples + f"\nReview: \"{review_text}\" ->"
    return prompt

def instruction_prompt(review_text):
    prompt = f"Analyze the sentiment of the following review: \"{review_text}\". Respond with 'positive' or 'negative'."
    return prompt

def generate_response(prompt, model, tokenizer, max_length=20):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_length=max_length, num_beams=5)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [24]:
output_file = 'flan_t5_imdb_results.txt'

with open(output_file, 'w') as f:
    for i, review in enumerate(sampled_reviews):
        review_text = review['text']
        true_label = review['labels']  # Assuming label is in dataset, 0=negative, 1=positive
        true_label_text = "positive" if true_label == 1 else "negative"
        
        # Zero-shot
        zero_shot = generate_response(zero_shot_prompt(review_text), flan_model, flan_tokenizer)
        
        # Few-shot
        few_shot = generate_response(few_shot_prompt(review_text), flan_model, flan_tokenizer)
        
        # Instruction-based
        instruction_based = generate_response(instruction_prompt(review_text), flan_model, flan_tokenizer)
        
        # Writing to file
        f.write(f"Review {i+1}: {review_text}\n")
        f.write(f"Review {i+1} true label: {true_label_text}\n")
        f.write(f"Review {i+1} zero-shot: {zero_shot}\n")
        f.write(f"Review {i+1} few-shot: {few_shot}\n")
        f.write(f"Review {i+1} instruction-based: {instruction_based}\n\n")


Token indices sequence length is longer than the specified maximum sequence length for this model (794 > 512). Running this sequence through the model will result in indexing errors
